In [10]:
import boto3
from boto3.s3.transfer import S3Transfer
import os
import import_ipynb
# from pdf2Image import convert_pdf
# from create_bucket import create_bucket,upload_file_s3,textract_job

In [11]:
!pip install pdf2Image

## Converting pdf2Images
The below function heps in converting pdf's to Images


In [12]:
from pdf2image import convert_from_path
import os

In [13]:
# Run this command -- ?
conda install -c conda-forge poppler

SyntaxError: invalid syntax (<ipython-input-13-781bfa385e5e>, line 2)

In [14]:
import os
print(os.getcwd())

C:\Users\lenovo\PycharmProjects\aws-textract1


In [26]:
# root directory
#conversion
def convert_pdf_Img(root_dir, save_dir, res=400):
    
    for pdf_path in os.listdir(root_dir):
        path = (root_dir +'\\' + pdf_path)
        pages = convert_from_path(path,res)

        name_with_extension = pdf_path.rsplit('/')[-1]
        name = name_with_extension.rsplit('.')[0]

        out_path = os.path.join(save_dir,name)
        os.makedirs(out_path)

        for idx,page in enumerate(pages):
            page.save(f'{out_path}/{name}_{idx}.png','PNG')

## Starting the Textract Job

In [41]:
# decision making whether to go for key-value or the table output
def textract_job(data_dir):
    s3_client = boto3.client('s3')
    tess = boto3.client('textract', region_name='ap-south-1')
    bucket = 'arthashastra-1'
    print(1)
    print("MAKE CHOICE FOR THE INPUT:")
    print("=============================")
    print("KV == KEY_VALUE PAIR \nTABLE == TABLE OUTPUT")
    
    inpt = input('enter the input : ')
    
    if inpt == 'KV':
        output_file = data_dir + '\\' + 'Output_KV.csv'
        path = data_dir + '\\' + 'Test_Inv'
        print(path)
        with open(output_file, "wt") as fout:
            files = os.listdir(path)
            for file in files:
                fle = os.listdir(path+'/'+file)
                for f in fle:
                    file_name = path+'/'+file+'/'+f
                    print(f)
                    main_KV(file_name,fout)
                    
    elif inpt=="TABLE":
        output_file = data_dir + '\\' + 'Output.csv'
        path = data_dir + '\\' + 'Test_Inv'
        with open(output_file, "wt") as fout:
            files = os.listdir(path)
            for file in files:
                fle = os.listdir(path+'/'+file)
                for f in fle:
                    file_name = path+'/'+file+'/'+f
                    print(f)
                    main(file_name,fout)
    else:
        print("****=============================****")
        print("please chooce the right option")
        textract_job(data_dir)
    

## Table Function

In [35]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint

In [36]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}

                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows

def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '
    return text

def get_table_csv_results(file_name):
    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks = response['Blocks']


    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index + 1)
        csv += '\n\n'

    return csv


def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)

    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():

        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'

    csv += '\n\n\n'
    return csv

def main(file_name,fout):
    table_csv = get_table_csv_results(file_name)

    #output_file = 'C:/Users/lenovo/PycharmProjects/aws-textract1/Output.csv'

    # replace content
    #fout.write(file_name)
    fout.write(table_csv)

    # show the results
    #print('CSV OUTPUT FILE: ', output_file)




## Key-Value pair Function

In [37]:
import boto3
import sys
import re
import json


In [38]:
def get_kv_map(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    client = boto3.client('textract')
    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['FORMS'])

    # Get the text blocks
    blocks=response['Blocks']
    

    # get key and value maps
    key_map = {}
    value_map = {}
    block_map = {}
    for block in blocks:
        block_id = block['Id']
        block_map[block_id] = block
        if block['BlockType'] == "KEY_VALUE_SET":
            if 'KEY' in block['EntityTypes']:
                key_map[block_id] = block
            else:
                value_map[block_id] = block

    return key_map, value_map, block_map


def get_kv_relationship(key_map, value_map, block_map):
    kvs = {}
    for block_id, key_block in key_map.items():
        value_block = find_value_block(key_block, value_map)
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        kvs[key] = val
    return kvs

def find_value_block(key_block, value_map):
    for relationship in key_block['Relationships']:
        if relationship['Type'] == 'VALUE':
            for value_id in relationship['Ids']:
                value_block = value_map[value_id]
    return value_block


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '    

                                
    return text

def print_kvs(kvs):
    for key, value in kvs.items():
        print(key, ":", value)


def search_value(kvs, search_key):
    for key, value in kvs.items():
        if re.search(search_key, key, re.IGNORECASE):
            return value

def main_KV(file_name,fout):

    key_map, value_map, block_map = get_kv_map(file_name)

    # Get Key Value relationship
    kvs = get_kv_relationship(key_map, value_map, block_map)
    print("\n\n== FOUND KEY : VALUE pairs ===\n")
    fout.write(str(kvs))
    #print_kvs(kvs)

    # Start searching a key value
    ''' while input('\n Do you want to search a value for a key? (enter "n" for exit) ') != 'n':
        search_key = input('\n Enter a search key:')
        print('The value is:', search_value(kvs, search_key)) '''


## Main Function

In [39]:
print(os.getcwd())

C:\Users\lenovo\PycharmProjects\aws-textract1


In [40]:

if __name__ == '__main__':
    
    #filename
    root_dir = os.getcwd()
    data_dir = root_dir + '\\' + 'data'
    print(data_dir)
    save_dir =  root_dir + '\\' + 'Images'
    # convert_pdf_Img(data_dir,save_dir,res=400)
    
    textract_job(root_dir)

C:\Users\lenovo\PycharmProjects\aws-textract1\data
1
MAKE CHOICE FOR THE INPUT:
KV == KEY_VALUE PAIR 
TABLE == TABLE OUTPUT
enter the inputfdmkv
****=============================****
please chooce the right option


TypeError: textract_job() missing 1 required positional argument: 'data_dir'